In [1]:
%pip install langchain-community 

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain.graphs import Neo4jGraph
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Phat121002@"

graph = Neo4jGraph()

C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_15176\382179826.py:7: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [3]:
import requests
import pandas as pd

# Define the URL and parameters
url = "https://datasets-server.huggingface.co/rows"
params = {
    "dataset": "Nicolybgs/healthcare_data",
    "config": "default",
    "split": "train",
    "offset": 0,
    "length": 100
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a Pandas DataFrame
    rows = data.get('rows', [])
    df = pd.DataFrame([row['row'] for row in rows])

In [17]:
import pandas as pd
from langchain.docstore.document import Document


# Define the function to format each row
def format_row(row):
    return (
        f"Available Extra Rooms in Hospital: {row['Available Extra Rooms in Hospital']}, "
        f"Department: {row['Department']}, Ward_Facility_Code: {row['Ward_Facility_Code']}, "
        f"Doctor Name: {row['doctor_name']}, Staff Available: {row['staff_available']}, "
        f"Patient ID: {row['patientid']}, Age: {row['Age']}, Gender: {row['gender']}, "
        f"Type of Admission: {row['Type of Admission']}, Severity of Illness: {row['Severity of Illness']}, "
        f"Health Conditions: {row['health_conditions']}, Visitors with Patient: {row['Visitors with Patient']}, "
        f"Insurance: {row['Insurance']}, Admission Deposit: {row['Admission_Deposit']}, "
        f"Stay (in days): {row['Stay (in days)']}\n\n"
    ).lower()

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

# Convert the formatted text into a list of Document objects
documents = []
for text in df['formatted_text'].head(5).tolist():
    document = Document(page_content=text)
    documents.append(document)

In [7]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(documents)

In [9]:
print("Documents after splitting:")
for doc in documents:
    print(doc.page_content)

Documents after splitting:
available extra rooms in hospital: 4, department: gynecology, ward_facility_code: d, doctor name: dr sophia, staff available: 0, patient id: 33070, age: 41-50, gender: female, type of admission: trauma, severity of illness: extreme, health conditions: diabetes, visitors with patient: 4, insurance: yes, admission deposit: 2966.408696, stay (in days): 8


available extra rooms in hospital: 4, department: gynecology, ward_facility_code: b, doctor name: dr sophia, staff available: 2, patient id: 34808, age: 31-40, gender: female, type of admission: trauma, severity of illness: minor, health conditions: heart disease, visitors with patient: 2, insurance: no, admission deposit: 3554.835677, stay (in days): 9


available extra rooms in hospital: 2, department: gynecology, ward_facility_code: b, doctor name: dr sophia, staff available: 8, patient id: 44577, age: 21-30, gender: female, type of admission: trauma, severity of illness: extreme, health conditions: diabete

In [10]:

from langchain_community.llms import Ollama

llm = Ollama(model="phi")
llm.invoke("Tell me a joke")

C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_15176\203874901.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi")


' AI '

In [18]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Khởi tạo transformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Chọn 20 documents đầu tiên
subset_documents = documents[:5]

# Chuyển sang graph documents
graph_documents = llm_transformer.convert_to_graph_documents(subset_documents)

# Đẩy lên Neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=False,   # hoặc True nếu bạn muốn thêm label '__Entity__'
    include_source=True      # thêm metadata gốc
)


In [20]:
%pip install transformers sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install -U transformers==4.36.2


  Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.2 MB 7.5 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.2 MB 10.7 MB/s eta 0:00:01
   ------------------------------ --------- 6.3/8.2 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 8.2/8.2 MB 10.6 MB/s eta 0:00:00
Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.0
    Uninstalling transformers-4.51.0:
      Successfully uninstalled transformers-4.51.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.0.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5" # hoặc "cpu"
)

vector = embeddings.embed_query("Test embedding")
print(vector[:10])  # In thử 10 phần tử đầu


[0.02927350625395775, -0.045802462846040726, -0.015977395698428154, 0.06701088696718216, 0.059911761432886124, 0.05131841078400612, 0.03220994397997856, -0.022907845675945282, 0.03211498633027077, -0.032036054879426956]


In [33]:

from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [34]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

In [35]:
%pip install gradio

   ---------------------------------------- 0.0/46.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/46.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/46.5 MB 1.7 MB/s eta 0:00:28
    --------------------------------------- 1.0/46.5 MB 2.3 MB/s eta 0:00:20
   - -------------------------------------- 1.8/46.5 MB 2.6 MB/s eta 0:00:17
   -- ------------------------------------- 2.4/46.5 MB 2.7 MB/s eta 0:00:17
   -- ------------------------------------- 3.1/46.5 MB 2.8 MB/s eta 0:00:16
   --- ------------------------------------ 4.5/46.5 MB 3.3 MB/s eta 0:00:13
   ---- ----------------------------------- 5.0/46.5 MB 3.4 MB/s eta 0:00:13
   ----- ---------------------------------- 6.0/46.5 MB 3.4 MB/s eta 0:00:12
   ------ --------------------------------- 7.3/46.5 MB 3.7 MB/s eta 0:00:11
   ------ --------------------------------- 8.1/46.5 MB 3.8 MB/s eta 0:00:11
   ------- -------------------------------- 8.9/46.5 MB 3.9 MB/s eta 0:00:10
   -------- -

In [37]:
df.head(5)  # In thử 5 dòng đầu tiên của DataFrame

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,Stay (in days),formatted_text
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,8,"available extra rooms in hospital: 4, departme..."
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,9,"available extra rooms in hospital: 4, departme..."
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,7,"available extra rooms in hospital: 2, departme..."
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,8,"available extra rooms in hospital: 4, departme..."
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,34,"available extra rooms in hospital: 2, departme..."


C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_15176\2836696987.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


In [36]:
import gradio as gr

# Define the function for querying patient details
def query_patient_details(query):
    try:
        result = qa_chain({"query": query})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=query_patient_details,        # Function to call
    inputs=gr.Textbox(label="Enter your question"),  # Input textbox
    outputs=gr.Textbox(label="Answer")   # Output textbox
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
